# Chiral solutions for Type-I Dirac seesaw

In [1]:
import pandas as pd
import numpy as np
import itertools
import sys
pd.set_option('display.max_colwidth',400)
pd.set_option('display.max_columns',100)

## Load full solutions

In [2]:
ds=pd.read_json('https://github.com/restrepo/anomaly/raw/main/solutions.json')
ds.shape

(148097, 5)

Check that solutions are unique

In [3]:
ds['strz']=ds['solution'].astype(str)
ds.drop_duplicates('strz').shape

(148097, 6)

In [4]:
ds=ds.drop('strz',axis='columns')

Number of solutions with repeated charges

In [5]:
ds[ds['solution'].apply(lambda l: len(l)-len(set(l)))>0].shape[0]

95358

## Check phenomenological conditions
$$ 2l+\nu-m=0$$

In [6]:
#See anomalysolutions.ipynb for details
#from anomalytools import *

### Prepare functions to filter solutions with at least two sets of repeated charges

In [7]:
def multiple_repeated(ll):
    MR=False
    rp=[]
    for x in ll:
        if ll.count(x)>1:
            rp.append(x)
    rp=sorted(np.unique(rp))
    if len(rp)>=2:
        MR=True
    return MR

def extract_Dirac_and_Majorana(l,s):
    ll=l.copy()
    #Check first Dirac
    for xs  in itertools.combinations(l,2):
        #print('ini',xs,ll,xs[0] in ll,xs[1] in ll)
        if (xs[0] in ll  and xs[1] in ll)  and (sum(xs)+s==0 or sum(xs)-s==0):
            ll=[x for x in ll if x in ll and x not in xs]
        #print('ll',ll)
    #Check Majorana
    for x in ll:
        if 2*x+s==0 or 2*x-s==0:
            ll.remove(x)
    return ll

def best_higgs_singlet(l):
    lenmax=np.Inf
    best_sp=0
    best_spl=l
    sps=np.unique( (np.abs( list(2*np.array(l))+[sum(x) for x in itertools.combinations(l,2) ]) ) )
    for sp in sps:
        newl=extract_Dirac_and_Majorana(l,sp)
        if len(newl)<len(best_spl):
            best_spl=newl
            best_sp=sp
    return best_sp,best_spl

def tree_level(ll):
    '''
    Given a solution find set of repeated charges and assign to `l` and `ν`, e.g
       ll=[l,l,...,ν,ν,...]
    For each combination of repeated sets calculates:
       m=2l+ν
    If `m ≠ l ≠ ν` in ll, returns a `N → N+4` solution
       [m,m,m,-m-m,l,l,...,ν,ν,...]
    else, returns a `N → N+6` solution
       [m,m,m,-m,-m,-m,l,l,...,ν,ν,...]
    '''
    ms=[]
    rp=[]
    for x in ll:
        if ll.count(x)==1:
            ms.append(x)
        elif ll.count(x)>1:
            rp.append(x)
    rp=sorted(np.unique(rp))
    #print(rp,isinstance(rp,list))
    #Two or more repeated sets
    sltn=[]
    if len(rp)>=2:
        rps=[k  for k  in itertools.combinations(rp,2)]
        #print('rps1',rps)
        rps=rps+[tuple(reversed(k)) for k  in rps]
        #print('rps2',rps)
        for r in rps:
            #Loop on combinatories here, break whit the first solution
            l=r[0]
            ν=r[1]
            m=2*l+ν
            s=l-m
            massless=ms.copy()
            massless=massless+[x for x in rp if x not in r and x not in massless]
            #print('m,s',massless,s,m,rp)
            #print('ini massless',massless)
            if m in ll and m not in rp:
                massless.remove(m)
                #Check Dirac and Majorana
                massless=extract_Dirac_and_Majorana(massless,s)
                slt={'[l,ν,s,m_in,m_out]':[l,ν,s,m,0],'massless':massless}
            else:
                massless=extract_Dirac_and_Majorana(massless,s)                
                slt={'[l,ν,s,m_in,m_out]':[l,ν,s,0,m],'massless':massless}
                
            if len(massless)>0:
                slt['sp'],slt['sp_massless']=best_higgs_singlet(massless)
            else:
                slt['sp'],slt['sp_massless']=(None,None)
            sltn.append(slt)
    return sltn

assert extract_Dirac_and_Majorana([2,6,4,3],8)==[3]
assert best_higgs_singlet([1,-11,9])==(2, [])
assert tree_level([1, -4, -4, 9, 9,7,7,7,-11])[0]['[l,ν,s,m_in,m_out]']==[-4, 7, -3, 0, -1]
assert tree_level([ -4, -4,-4, 9, 9,9])[0]['sp']==None
assert not tree_level([1, 1, -3, -4, 6, 6, -7])[0]['sp_massless']

### Initialize filtered solutions

In [266]:
cl=ds.copy()
cl=cl[cl['solution'].apply(multiple_repeated)].reset_index(drop=True)

Find solutions that satisfy the condition

In [267]:
cl['tree_level']=cl['solution'].apply(tree_level)

In [268]:
def extract_massless(ll,tl):
    tln=[]
    for dd in tl:
        d=dd.copy()
        dp=dict(zip( ['l','ν','s','m_in','m_out'], d['[l,ν,s,m_in,m_out]'] ))
        Maj=0
        l=len([x for x in ll if x==dp['l']])
        dp['r']=dp['s']-dp['l']
        if dp['m_in']:        
            r=2
        if dp['m_out']:
            r=3
        if not dp['m_in'] and not dp['m_out']:
            r=l 
            Maj=0 # Go to 1
        if l>r:
            Maj=dp['l']
        elif l<r:
            Maj=dp['r']
        if Maj and not d['massless'] and d['sp'] is None:
            #print(Maj)
            d['sp']=Maj*2
            d['massless']=[Maj]
            d['sp_massless']=[]
            d['rank']=0
        tln.append(d) #1
    return tln

In [269]:
cl.shape

(31997, 7)

In [ ]:
cl[cl['solution'].astype(str)=='[1, 1, -2, 4, 5, -7, -7, 8]']

In [270]:
cl['tree_level']=cl.apply(lambda row: extract_massless(row['solution'],row['tree_level'])  ,axis='columns')

Reorder the filtered solutions

In [271]:
cl['smax']=cl['solution'].apply(lambda l:np.abs(l).max())
cl=cl[cl['tree_level'].apply(lambda l: [d for d in l if len(d.get('massless'))==0 or len(d.get('sp_massless'))==0 ] if isinstance(l,list) else None 
               ).apply(len)>0].sort_values(['n','smax']).reset_index(drop=True)

cl['tree_level']=cl['tree_level'].apply(lambda l: [d for d in l if len(d.get('massless'))==0 or 
                                                   len(d.get('sp_massless'))==0 ] if isinstance(l,list) else None 
             )
cl=cl[cl.smax<=30].reset_index(drop=True)
cl[:1]

,l,k,solution,gcd,n,sltn,tree_level,smax
0,"[-1, -2]","[-1, 2]","[1, 1, 1, -4, -4, 5]",1,6,"[1, 1, 1, -4, -4, 5]","[{'[l,ν,s,m_in,m_out]': [-4, 1, 3, 0, -7], 'massless': [5], 'sp': 10, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [1, -4, 3, 0, -2], 'massless': [5], 'sp': 10, 'sp_massless': []}]",5


In [272]:
cl.shape

(6385, 8)

### Flag equivalent solutions

In [273]:
def characterize_solution(cl,i,j=0):
    if j%2==0:
        cl.loc[i,'D_msls']=j
    else:
        cl.loc[i,'M_msls']=j
    #if j==0:
    #    cl.loc[i,'M_msls']=j
        
    cl.loc[i,'rptd']=len( [ cl.loc[i,'solution'].count(x) for x in np.unique(cl.loc[i,'solution']) if cl.loc[i,'solution'].count(x)>1] )
    sl=[d.get('[l,ν,s,m_in,m_out]') for d in cl.loc[i,'tree_level'] if len(d.get('massless'))==j]
    if len(sl)>0:
        cl.loc[i,'nl']=cl.loc[i,'solution'].count(sl[0][0] )
        cl.loc[i,'nν'] =cl.loc[i,'solution'].count(sl[0][1] )
    if 0 in [l[3] for l in sl]:
        cl.loc[i,'nr']=3
    else:
        cl.loc[i,'nr']=2
    cl.loc[i,'rk']=[d.get('rank') for d in cl.loc[i,'tree_level']][0]
    return cl

In [274]:
for i in cl[cl['tree_level'].apply(lambda l: [d for d in l if len(d.get('massless'))==0 ] if isinstance(l,list) else None 
             ).apply(len)>0].index:
    cl=characterize_solution(cl,i,0)

Example: massless solutions

In [278]:
cl.dropna()[:5]

,l,k,solution,gcd,n,sltn,tree_level,smax,D_msls,rptd,nl,nν,nr,rk
37,"[0, -1, 0]","[-2, -1, -2, -1]","[1, 4, 5, 5, -8, -10, -10, 13]",1,8,"[1, 4, 5, 5, -8, -10, -10, 13]","[{'[l,ν,s,m_in,m_out]': [-10, 5, 5, 0, -15], 'massless': [15], 'sp': 30, 'sp_massless': [], 'rank': 0}, {'[l,ν,s,m_in,m_out]': [5, -10, 5, 0, 0], 'massless': [], 'sp': None, 'sp_massless': None}]",13,0.0,2.0,2.0,2.0,3.0,0.0
278,"[1, 2, 1, -1]","[-6, -3, -5, -3]","[1, 2, -6, -6, -6, 8, 9, 9, -11]",1,9,"[1, 2, -6, -6, -6, 8, 9, 9, -11]","[{'[l,ν,s,m_in,m_out]': [-6, 9, -3, 0, -3], 'massless': [], 'sp': None, 'sp_massless': None}, {'[l,ν,s,m_in,m_out]': [9, -6, -3, 0, 12], 'massless': [-12], 'sp': -24, 'sp_massless': [], 'rank': 0}]",11,0.0,2.0,3.0,2.0,3.0,1.0
281,"[3, -2, 3]","[-2, -1, -2, 4]","[4, 4, 4, -5, -9, -10, -10, 11, 11]",1,9,"[4, 4, 4, -5, -9, -10, -10, 11, 11]","[{'[l,ν,s,m_in,m_out]': [-10, 4, 6, 0, -16], 'massless': [-9], 'sp': 18, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [-10, 11, -1, -9, 0], 'massless': [], 'sp': None, 'sp_massless': None}, {'[l,ν,s,m_in,m_out]': [4, 11, -15, 0, 19], 'massless': [-9], 'sp': 18, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [4, -10, 6, 0, -2], 'massless': [-9], 'sp': 18, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [1...",11,0.0,3.0,2.0,2.0,2.0,1.0
306,"[-2, -1, 0]","[-1, -3, -5, -3]","[5, 6, 6, 6, -8, -9, -9, -10, 13]",2,9,"[5, 6, 6, 6, -8, -9, -9, -10, 13]","[{'[l,ν,s,m_in,m_out]': [-9, 6, 3, 0, -12], 'massless': [12], 'sp': 24, 'sp_massless': [], 'rank': 0}, {'[l,ν,s,m_in,m_out]': [6, -9, 3, 0, 3], 'massless': [], 'sp': None, 'sp_massless': None}]",13,0.0,2.0,3.0,2.0,3.0,0.0
341,"[-2, -1, 0]","[-1, -4, -5, -4]","[1, -5, 8, 8, 8, -11, -12, -12, 15]",2,9,"[1, -5, 8, 8, 8, -11, -12, -12, 15]","[{'[l,ν,s,m_in,m_out]': [-12, 8, 4, 0, -16], 'massless': [16], 'sp': 32, 'sp_massless': [], 'rank': 0}, {'[l,ν,s,m_in,m_out]': [8, -12, 4, 0, 4], 'massless': [], 'sp': None, 'sp_massless': None}]",15,0.0,2.0,3.0,2.0,3.0,0.0


### Add solutions with massless fermions that get masses with some $S'$

In [279]:
for j in range(1,6):
    print(j)
    for i in cl[cl['tree_level'].apply(lambda l: [d for d in l if d.get('sp_massless') is not None and len(d.get('sp_massless'))==0  and len(d.get('massless'))==j ] if isinstance(l,list) else None 
             ).apply(len)>0].index:
        cl=characterize_solution(cl,i,j)

1
2
3
4
5


In [283]:
cl['rk']=cl['rk'].fillna(1)

In [284]:
cl=cl[cl['n']<10].reset_index(drop=True)

In [286]:
cl['D_msls']=cl['D_msls'].apply(lambda n: 10 if np.isnan(n) else n)
cl['M_msls']=cl['M_msls'].apply(lambda n: 10 if np.isnan(n) else n)
for k  in ['D_msls','M_msls','nl','nν','nr','rptd','rk']:
    cl[k]=cl[k].astype(int)

In [287]:
cl=cl.sort_values(['n','smax','D_msls','rptd','nl','nν','nr']).reset_index(drop=True)

In [288]:
cl['sltn']=cl['solution'].astype(str)
ds['sltn']=ds['solution'].astype(str)

In [289]:
cl[cl['solution'].astype(str)=='[1, -2, -2, 4, 5, -7, -7, 8]']#.apply(lambda row: extract_massless(row['solution'],row['tree_level'])  ,axis='columns')

,l,k,solution,gcd,n,sltn,tree_level,smax,D_msls,rptd,nl,nν,nr,rk,M_msls
21,"[-1, 0, 1]","[-4, -2, 2]","[1, -2, -2, 4, 5, -7, -7, 8]",1,8,"[1, -2, -2, 4, 5, -7, -7, 8]","[{'[l,ν,s,m_in,m_out]': [-7, -2, 9, 0, -16], 'massless': [16], 'sp': 32, 'sp_massless': [], 'rank': 0}, {'[l,ν,s,m_in,m_out]': [-2, -7, 9, 0, -11], 'massless': [11], 'sp': 22, 'sp_massless': [], 'rank': 0}]",8,10,2,2,2,3,0,1


In [310]:
cl[cl['solution'].astype(str)=='[1, 1, 2, 2, 4, -5, -7, -7, 9]']

,l,k,solution,gcd,n,sltn,tree_level,smax,D_msls,rptd,nl,nν,nr,rk,M_msls
268,"[1, 2, 1, 2]","[-5, -3, -6, -3]","[1, 1, 2, 2, 4, -5, -7, -7, 9]",1,9,"[1, 1, 2, 2, 4, -5, -7, -7, 9]","[{'[l,ν,s,m_in,m_out]': [-7, 1, 6, 0, -13], 'massless': [-5, 9], 'sp': 4, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [-7, 2, 5, 0, -12], 'massless': [-5, 9], 'sp': 4, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [1, -7, 6, -5, 0], 'massless': [9], 'sp': 18, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [2, -7, 5, 0, -3], 'massless': [-5, 9], 'sp': 4, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [2, 1...",9,2,3,2,2,3,1,1


## Build the final table dropping out equivalent solutions 

In [331]:
fl.shape

(68, 15)

In [325]:
dk.get('[l,ν,s,m_in,m_out]')

[11, -15, 4, 7, 0]

In [705]:
fl=cl.copy()

In [927]:
cl.shape

(986, 15)

In [1022]:
fl=cl.copy()

In [1023]:
def rank(l,massless,s):
    '''
    Return 0 if the integers in massless appears as s-paired same-dimension multiplets in l, 1 otherwise.
    s-paired means that exist two intengers in massless such that n1+n2=±s. (n1 can be equal to n2)
    
    For example, for l containing only the massless=[1,2] integers:
        l=[1,2]; s=3 → 0: 3-paired multiplet: ([1],[2]) of 1-dimension multiplets (A.K.A singlet)
        l=[1,1,2]; s=3 → 1: unpaired multiplets: ([1,1],[2]) doblet and singlet
        l=[1,1,2,2]; s=3 → 0: 3-paired multiplets: ([1,1],[2]) doblet and singlet
        l=[1,1,2,2,2]; s=3 → 1: unpaired multiplets: ([1,1,1],[2,2]) doblet and singlet
    massless=[1,2,3]
    l=[1,2,3]; s=4 → 0: 4-paired multiplet: ([1],[3]) of singlets
                            4-paired multiplet: ([1],[3]) of singlet
    '''
    #Keep massless
    l=[x for x in  l if x in massless ]
    #Remove Majorana
    xx=(np.unique(l)*2-s)
    xx=(xx[xx!=0]+s)/2
    xx=(np.unique(xx)*2+s)
    xx=(xx[xx!=0]-s)/2
    #Get s-multiplets
    xxx=[l.count(x) for x in xx if l.count(x)!=1] 
    if len(xxx)%2==0: # s-multiplets must be paired
        # Check that each mutiplet is paired
        if 1 in [xxx.count(x)%2 for x in set(xxx)]:
            return 1
        else:
            #list of only 0's → if dimension of multiplets are paired
            return 0 # 
    else:
        return 1
        

def extract_s_massless(row):
    lr=[]
    for d in row['tree_level']:
        dp=dict(zip( ['l','ν','s','m_in','m_out'], d['[l,ν,s,m_in,m_out]'] ))
        #print(dp,d['massless'],len(d['massless'])==1,d['massless'][0],-dp['m_in'])
        if (not d['massless'] 
            or (len(d['massless'])==1 and (d['massless'][0]==-dp['m_in']  or d['massless'][0]==dp['l'] ) )
            or (len(d['massless'])==1 and (d['massless'][0]==-dp['m_out'] or d['massless'][0]==dp['l'] ) )
               ):
            massless=[x for x in row['solution'] if x!=dp['l'] and x!=dp['ν'] and x!=dp['m_in'] and x!=dp['m_out']]
            if rank(row['solution'],massless,dp['s'])==1:
                continue #discard this solution
        lr.append(d)
    return lr
    

assert rank([1,2],[1,2],3)==0
assert rank([1,1,2,2,2],[1,2],3)==1
assert rank([1,2,3],[1,2,3],4)==0
assert rank([1, 1, -4, 9, -10, -10, 16, 16, -19],[9, -19, 1],18)==1
assert rank( [1, -4, -4, 9, 9, -11], [-11],22)==0
assert rank( [-16,-16,-16, 17, 17],[-16, 17],1)==1
assert rank( [-16,-16,-16, 17, 17,-18,-18,19,19,19],[-16, 17,-18,19],1)==0

assert extract_s_massless( {'solution':[4, -5, -9, -10, -10, 11, 11],
                            'tree_level':[{'[l,ν,s,m_in,m_out]': [-10, 11, -1, -9, 0], 
                            'massless': [], 'sp': None, 'sp_massless': None}]}
                  )[0]['[l,ν,s,m_in,m_out]']==[-10, 11, -1, -9, 0]
assert extract_s_massless( {'solution':[4, 4, 4, -5, -9, -10, -10, 11, 11],
                            'tree_level':[{'[l,ν,s,m_in,m_out]': [-10, 11, -1, -9, 0], 
                            'massless': [], 'sp': None, 'sp_massless': None}]} )==[]
assert extract_s_massless( {'solution':[4, 4, 4, -5, -9, -10, -10, -10, 11, 11],
                            'tree_level':[{'[l,ν,s,m_in,m_out]': [-10, 11, -1, -9, 0], 
                            'massless': [-10], 'sp': 20, 'sp_massless': []}]} )==[]
assert extract_s_massless( {'solution':[4, 4, 4, -5, -10, -10, 11, 11],
                            'tree_level':[{'[l,ν,s,m_in,m_out]': [-10, 11, -1, 0, -9], 
                            'massless': [9], 'sp': 18, 'sp_massless': []}]} 
                  )==[]

In [1024]:
fl['tree_level']=fl.apply(extract_s_massless,axis='columns')

In [1025]:
fl[fl['solution'].astype(str)=='[3, -4, -6, -6, 7, 7, 8, -9]']

,l,k,solution,gcd,n,sltn,tree_level,smax,D_msls,rptd,nl,nν,nr,rk,M_msls
23,"[0, -3, -1]","[-3, -5, -6, -4]","[3, -4, -6, -6, 7, 7, 8, -9]",1,8,"[3, -4, -6, -6, 7, 7, 8, -9]","[{'[l,ν,s,m_in,m_out]': [-6, 7, -1, 0, -5], 'massless': [5], 'sp': 10, 'sp_massless': [], 'rank': 0}, {'[l,ν,s,m_in,m_out]': [7, -6, -1, 8, 0], 'massless': [-9], 'sp': 18, 'sp_massless': []}]",9,10,2,2,2,3,0,1


In [1031]:
fl[fl['solution'].astype(str)=='[5, 6, 6, 6, -8, -9, -9, -10, 13]']

,l,k,solution,gcd,n,sltn,tree_level,smax,D_msls,rptd,nl,nν,nr,rk,M_msls
301,"[-2, -1, 0]","[-1, -3, -5, -3]","[5, 6, 6, 6, -8, -9, -9, -10, 13]",2,9,"[5, 6, 6, 6, -8, -9, -9, -10, 13]","[{'[l,ν,s,m_in,m_out]': [-9, 6, 3, 0, -12], 'massless': [12], 'sp': 24, 'sp_massless': [], 'rank': 0}, {'[l,ν,s,m_in,m_out]': [6, -9, 3, 0, 3], 'massless': [], 'sp': None, 'sp_massless': None}]",13,0,2,2,3,3,0,1


In [1029]:
fl[fl['solution'].astype(str)=='[4, 4, 4, -5, -9, -10, -10, 11, 11]']

,l,k,solution,gcd,n,sltn,tree_level,smax,D_msls,rptd,nl,nν,nr,rk,M_msls
277,"[3, -2, 3]","[-2, -1, -2, 4]","[4, 4, 4, -5, -9, -10, -10, 11, 11]",1,9,"[4, 4, 4, -5, -9, -10, -10, 11, 11]","[{'[l,ν,s,m_in,m_out]': [-10, 4, 6, 0, -16], 'massless': [-9], 'sp': 18, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [4, 11, -15, 0, 19], 'massless': [-9], 'sp': 18, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [4, -10, 6, 0, -2], 'massless': [-9], 'sp': 18, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [11, -10, -1, 0, 12], 'massless': [-9], 'sp': 18, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [11,...",11,0,3,2,3,3,1,1


In [933]:
def getit(it,dk):
    it['ll']=dk.get('[l,ν,s,m_in,m_out]')[0]
    it['ν']=dk.get('[l,ν,s,m_in,m_out]')[1]
    it['s']=dk.get('[l,ν,s,m_in,m_out]')[2]
    it['massless']=dk.get('massless')
    l=it['solution'].copy()    
    nν=l.count(it['ν'])
    ll=l.count(it['ll'])
    if dk.get('[l,ν,s,m_in,m_out]')[3]!=0:
        m=dk.get('[l,ν,s,m_in,m_out]')[3]
        if ll==2 or -m in it['massless']:#True:#nν==2:# and ll==2:
            it['extra']=[m,m,-m,-m]
        else:
            it['extra']=None
    else:
        m=dk.get('[l,ν,s,m_in,m_out]')[4]
        if ll==3  or -m in it['massless']:
            it['extra']=[m,m,m,-m,-m,-m]
        else:
            it['extra']=None            
    it['sp']=dk.get('sp')
    return it
    
tm=pd.DataFrame()

fl=fl.sort_values(['n','smax','D_msls','M_msls']).reset_index(drop=True)


for i in fl.index:
    it=fl.loc[i].to_dict()
    if fl.loc[i,'D_msls']==10:
        k=[d for d in fl.loc[i,'tree_level'] if len(d.get('massless'))==fl.loc[i,'M_msls']]
        kk=[d for d in  k if d.get('[l,ν,s,m_in,m_out]')[3]!=0]
        if len(k)>0:
            if len(kk)>0:
                dk=kk[0]
            else:
                dk=k[0]
            it=getit(it,dk)
            if it['extra']:
                it['nn']=it['n']+len(it['extra'])
            tm=tm.append(it,ignore_index=True)
    elif fl.loc[i,'M_msls']==10:
        k=[d for d in fl.loc[i,'tree_level'] if len(d.get('massless'))==fl.loc[i,'D_msls']]
        kk=[d for d in  k if d.get('[l,ν,s,m_in,m_out]')[3]!=0]
        if len(k)>0:
            if len(kk)>0:
                dk=kk[0]
            else:
                dk=k[0]
            dk=k[0]
            it=getit(it,dk)
            if it['extra']:
                it['nn']=it['n']+len(it['extra'])            
            tm=tm.append(it,ignore_index=True)
    else:
        k=[d for d in fl.loc[i,'tree_level'] if len(d.get('massless'))==fl.loc[i,'M_msls']]
        kk=[d for d in  k if d.get('[l,ν,s,m_in,m_out]')[3]!=0]
        if len(k)>0:
            if len(kk)>0:
                dk=kk[0]
            else:
                dk=k[0]
            dk=k[0]
            it=getit(it,dk)
            if it['extra']:            
                it['nn']=it['n']+len(it['extra'])            
            tm=tm.append(it,ignore_index=True)
        k=[d for d in fl.loc[i,'tree_level'] if len(d.get('massless'))==fl.loc[i,'D_msls']]
        kk=[d for d in  k if d.get('[l,ν,s,m_in,m_out]')[3]!=0]
        if len(k)>0:
            if len(kk)>0:
                dk=kk[0]
            else:
                dk=k[0]
            dk=k[0]
            it=getit(it,dk)
            if it['extra']:            
                it['nn']=it['n']+len(it['extra'])            
            tm=tm.append(it,ignore_index=True)
    #if i==26:
    #    break

In [934]:
#assert tm[tm['solution'].astype(str)=='[3, -4, -6, -6, 7, 7, 8, -9]'].reset_index(drop=True).loc[0,'ν']==-6

In [935]:
tm[tm['solution'].astype(str)=='[1, 1, 2, 2, 4, -5, -7, -7, 9]']

,D_msls,M_msls,extra,gcd,k,l,ll,massless,n,nl,nr,nν,rk,rptd,s,sltn,smax,solution,sp,tree_level,ν,nn
278,2.0,1.0,"[-5, -5, 5, 5]",1.0,"[-5, -3, -6, -3]","[1, 2, 1, 2]",1.0,[9],9.0,2.0,3.0,2.0,1.0,3.0,6.0,"[1, 1, 2, 2, 4, -5, -7, -7, 9]",9.0,"[1, 1, 2, 2, 4, -5, -7, -7, 9]",18,"[{'[l,ν,s,m_in,m_out]': [-7, 1, 6, 0, -13], 'massless': [-5, 9], 'sp': 4, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [-7, 2, 5, 0, -12], 'massless': [-5, 9], 'sp': 4, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [1, -7, 6, -5, 0], 'massless': [9], 'sp': 18, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [2, -7, 5, 0, -3], 'massless': [-5, 9], 'sp': 4, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [2, 1...",-7.0,13.0
279,2.0,1.0,None,1.0,"[-5, -3, -6, -3]","[1, 2, 1, 2]",-7.0,"[-5, 9]",9.0,2.0,3.0,2.0,1.0,3.0,6.0,"[1, 1, 2, 2, 4, -5, -7, -7, 9]",9.0,"[1, 1, 2, 2, 4, -5, -7, -7, 9]",4,"[{'[l,ν,s,m_in,m_out]': [-7, 1, 6, 0, -13], 'massless': [-5, 9], 'sp': 4, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [-7, 2, 5, 0, -12], 'massless': [-5, 9], 'sp': 4, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [1, -7, 6, -5, 0], 'massless': [9], 'sp': 18, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [2, -7, 5, 0, -3], 'massless': [-5, 9], 'sp': 4, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [2, 1...",1.0,13.0


In [936]:
tm=tm[tm.apply(lambda row: rank(row['solution'],row['massless'],row['sp']) if row['sp'] else 0,axis='columns')==0].reset_index(drop=True)

In [937]:
tm[tm['solution'].astype(str)=='[4, 4, 4, -5, -9, -10, -10, 11, 11]']

,D_msls,M_msls,extra,gcd,k,l,ll,massless,n,nl,nr,nν,rk,rptd,s,sltn,smax,solution,sp,tree_level,ν,nn
281,0.0,1.0,None,1.0,"[-2, -1, -2, 4]","[3, -2, 3]",-10.0,[-9],9.0,2.0,3.0,3.0,1.0,3.0,6.0,"[4, 4, 4, -5, -9, -10, -10, 11, 11]",11.0,"[4, 4, 4, -5, -9, -10, -10, 11, 11]",18,"[{'[l,ν,s,m_in,m_out]': [-10, 4, 6, 0, -16], 'massless': [-9], 'sp': 18, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [4, 11, -15, 0, 19], 'massless': [-9], 'sp': 18, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [4, -10, 6, 0, -2], 'massless': [-9], 'sp': 18, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [11, -10, -1, 0, 12], 'massless': [-9], 'sp': 18, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [11,...",4.0,NaN


In [938]:
tm=tm.dropna(subset=['extra']).reset_index(drop=True)

In [939]:
tm.shape

(336, 22)

In [940]:
tm.shape

(336, 22)

In [941]:
tm['nnn']=(tm['extra'].apply(len)/2).astype(int)+tm['n']

In [942]:
for k in ['nnn','gcd','ll','ν','s','n','D_msls','rptd','nl','nν','nr','M_msls']:
    tm[k]=tm[k].astype(int)
tm['nsmax']=tm.apply(lambda row: np.abs(row['extra']+row['solution']).max(),axis='columns' )
tm['lextra']=tm['extra'].apply(len)
tm=tm.sort_values(['nnn','n','nsmax']).reset_index(drop=True)

#Delete equivalent solutions
tm['Q']=True
sltns={#'[3, -4, 8, -9]': [3, -4, -6, -6, 7, 7, 8, -9],
       #'[1, -4, -9]':[1, -4, 5, 5, -9, -9, -9, 10, 10],
       #'[1, -5, 8, -17, 21]':[1, -5, 8, 12, 12, -16, -16, -17, 21],
       # '[25, 29]': [9, 9, -24, -24, -24, 25, 29],
       # '[13, 23]':[5, 5, 5, 13, -17, -17, -17, 23]#,
       #'[-4, 6, -10, 12]':[3, 3, -4, 6, -10, -11, -11, 12, 12]       
      }
for k in sltns.keys():
    iq=tm[((tm['solution'].astype(str)==str(sltns[k])) &  (tm['massless'].astype(str)==k)
          )].index[0] #unstable dark matter
    tm.loc[iq,'Q']=False
#alltm=tm.copy()
tm=tm[tm['Q']].reset_index(drop=True).drop('Q',axis='columns')
tm[['nnn','n','l','k','extra','solution','gcd','ll','ν','s','massless','sp','M_msls','rptd','nl','nν','nr','D_msls']
  ].sort_values(['nnn','M_msls','rptd','nl','nν','nr']).reset_index(drop=True)#.loc[[0,4]]#.loc[[0,4]]

,nnn,n,l,k,extra,solution,gcd,ll,ν,s,massless,sp,M_msls,rptd,nl,nν,nr,D_msls
0,8,6,"[1, -2]","[-4, 1]","[1, 1, -1, -1]","[1, -4, -4, 9, 9, -11]",3,-4,9,-5,[-11],22,1,2,2,2,3,2
1,9,7,"[-1, 1]","[-1, 0, -1]","[1, 1, -1, -1]","[1, 2, 2, -3, -3, -3, 4]",1,2,-3,1,[4],8,1,2,3,2,3,2
2,9,7,"[-3, -1]","[-2, -3, 1]","[-7, -7, 7, 7]","[3, 3, 3, -5, -5, -7, 8]",1,-5,3,2,[8],16,1,2,3,2,3,2
3,9,7,"[3, 1]","[-1, -5, 7]","[-4, -4, 4, 4]","[2, 2, -4, 7, -8, -8, 9]",1,2,-8,6,"[7, 9]",16,10,2,2,2,2,2
4,10,8,"[0, -3, -1]","[-3, -5, -6, -4]","[8, 8, -8, -8]","[3, -4, -6, -6, 7, 7, 8, -9]",1,7,-6,-1,[-9],18,1,2,2,2,3,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,12,9,"[8, -9, 8]","[-5, -3, -5, 9]","[-19, -19, -19, 19, 19, 19]","[9, 10, 22, -24, -24, -24, -27, 29, 29]",64,-24,29,-5,"[9, 10]",19,10,2,3,2,3,2
332,12,9,"[-6, -4, 3, -1]","[-2, -4, 3, -5]","[30, 30, 30, -30, -30, -30]","[6, 6, 6, -8, -13, -14, 18, 18, -19]",7,6,18,-24,"[-8, -13, -14, -19]",27,10,2,3,2,3,4
333,12,9,"[-6, -5, -4, -3]","[0, -5, -4, 1]","[-26, -26, -26, 26, 26, 26]","[6, 16, 16, 20, -21, -21, -21, -25, 30]",30,-21,16,5,"[6, 30]",36,10,2,3,2,3,2
334,12,9,"[-4, -1, -4]","[-3, -9, -3, -8]","[-38, -38, -38, 38, 38, 38]","[8, 8, -14, 18, 20, -23, -23, -23, 29]",3,-23,8,15,"[18, 20]",38,10,2,3,2,3,2


In [943]:
tm['nnn'].unique()

array([ 8,  9, 10, 11, 12])

In [944]:
tm['ex']=tm['extra'].apply(len)

In [945]:
for k  in ['D_msls','M_msls','nl','nν','nr','rptd','rk','ex']:
    tm[k]=tm[k].astype(int)

In [946]:
tm=tm.sort_values(['nnn','smax','M_msls','rptd','nl','nν','nr'])

In [947]:
fl=pd.DataFrame()

for nnn in tm['nnn'].unique():
    fl=fl.append( tm[ tm['nnn']==nnn ].drop_duplicates(subset=['D_msls','rptd','nl','nν','nr','M_msls','rk','ex']).reset_index(drop=True) )
fl=fl.reset_index(drop=True)

In [948]:
tm[tm['solution'].astype(str)=='[3, -4, -6, -6, 7, 7, 8, -9]']

,D_msls,M_msls,extra,gcd,k,l,ll,massless,n,nl,nr,nν,rk,rptd,s,sltn,smax,solution,sp,tree_level,ν,nn,nnn,nsmax,lextra,ex
7,10,1,"[8, 8, -8, -8]",1,"[-3, -5, -6, -4]","[0, -3, -1]",7,[-9],8,2,3,2,0,2,-1,"[3, -4, -6, -6, 7, 7, 8, -9]",9.0,"[3, -4, -6, -6, 7, 7, 8, -9]",18,"[{'[l,ν,s,m_in,m_out]': [-6, 7, -1, 0, -5], 'massless': [5], 'sp': 10, 'sp_massless': [], 'rank': 0}, {'[l,ν,s,m_in,m_out]': [7, -6, -1, 8, 0], 'massless': [-9], 'sp': 18, 'sp_massless': []}]",-6,12.0,10,9,4,4


In [949]:
tmo=tm.copy()

In [950]:
tmo[tmo['solution'].astype(str)=='[1, -2, -2, 4, 5, -7, -7, 8]'][ ['D_msls','rptd','nl','nν','nr','M_msls','rk','ex']]

,D_msls,rptd,nl,nν,nr,M_msls,rk,ex
32,10,2,2,2,3,1,0,6


In [951]:
for r in ['D_msls','rptd','nl','nν','nr','M_msls','rk','ex']:
    print(f"(qq['{r}']==) & ",end='')

(qq['D_msls']==) & (qq['rptd']==) & (qq['nl']==) & (qq['nν']==) & (qq['nr']==) & (qq['M_msls']==) & (qq['rk']==) & (qq['ex']==) & 

In [952]:
qq=tmo[ tmo['nnn']==10 ]
qq[( (qq['D_msls']==10)    )]

,D_msls,M_msls,extra,gcd,k,l,ll,massless,n,nl,nr,nν,rk,rptd,s,sltn,smax,solution,sp,tree_level,ν,nn,nnn,nsmax,lextra,ex
7,10,1,"[8, 8, -8, -8]",1,"[-3, -5, -6, -4]","[0, -3, -1]",7,[-9],8,2,3,2,0,2,-1,"[3, -4, -6, -6, 7, 7, 8, -9]",9.0,"[3, -4, -6, -6, 7, 7, 8, -9]",18,"[{'[l,ν,s,m_in,m_out]': [-6, 7, -1, 0, -5], 'massless': [5], 'sp': 10, 'sp_massless': [], 'rank': 0}, {'[l,ν,s,m_in,m_out]': [7, -6, -1, 8, 0], 'massless': [-9], 'sp': 18, 'sp_massless': []}]",-6,12.0,10,9,4,4
10,10,1,"[13, 13, -13, -13]",2,"[-3, -1, -3]","[1, -1, 1]",11,[-15],8,2,3,2,0,2,-2,"[1, -3, -9, -9, 11, 11, 13, -15]",15.0,"[1, -3, -9, -9, 11, 11, 13, -15]",30,"[{'[l,ν,s,m_in,m_out]': [-9, 11, -2, 0, -7], 'massless': [7], 'sp': 14, 'sp_massless': [], 'rank': 0}, {'[l,ν,s,m_in,m_out]': [11, -9, -2, 13, 0], 'massless': [-15], 'sp': 30, 'sp_massless': []}]",-9,12.0,10,15,4,4
11,10,1,"[1, 1, -1, -1]",4,"[-11, -7, 7]","[1, 0, -1]",7,[17],8,2,3,2,0,2,6,"[1, 5, 7, 7, -11, -13, -13, 17]",17.0,"[1, 5, 7, 7, -11, -13, -13, 17]",34,"[{'[l,ν,s,m_in,m_out]': [-13, 7, 6, 0, -19], 'massless': [19], 'sp': 38, 'sp_massless': [], 'rank': 0}, {'[l,ν,s,m_in,m_out]': [7, -13, 6, 1, 0], 'massless': [17], 'sp': 34, 'sp_massless': []}]",-13,12.0,10,17,4,4
13,10,1,"[-19, -19, 19, 19]",2,"[-11, -8, 8]","[-1, 0, 1]",-16,[22],8,2,3,2,1,2,3,"[1, 2, 13, 13, -16, -16, -19, 22]",22.0,"[1, 2, 13, 13, -16, -16, -19, 22]",44,"[{'[l,ν,s,m_in,m_out]': [-16, 13, 3, -19, 0], 'massless': [22], 'sp': 44, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [13, -16, 3, 0, 10], 'massless': [-10], 'sp': -20, 'sp_massless': [], 'rank': 0}]",13,12.0,10,22,4,4
16,10,1,"[4, 4, -4, -4]",1,"[-1, -6, -8, -6]","[0, -1, 0]",11,[3],8,2,3,2,0,2,7,"[3, 4, 11, 11, -17, -18, -18, 24]",24.0,"[3, 4, 11, 11, -17, -18, -18, 24]",6,"[{'[l,ν,s,m_in,m_out]': [-18, 11, 7, 0, -25], 'massless': [25], 'sp': 50, 'sp_massless': [], 'rank': 0}, {'[l,ν,s,m_in,m_out]': [11, -18, 7, 4, 0], 'massless': [3], 'sp': 6, 'sp_massless': []}]",-18,12.0,10,24,4,4
20,10,1,"[1, 1, -1, -1]",8,"[-11, -5, -11]","[1, 3, 2]",-11,[-13],8,2,3,2,1,2,-12,"[1, -11, -11, -13, 17, 23, 23, -29]",29.0,"[1, -11, -11, -13, 17, 23, 23, -29]",26,"[{'[l,ν,s,m_in,m_out]': [-11, 23, -12, 1, 0], 'massless': [-13], 'sp': 26, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [23, -11, -12, 0, 35], 'massless': [-35], 'sp': -70, 'sp_massless': [], 'rank': 0}]",23,12.0,10,29,4,4


In [953]:
tm=fl.copy()

In [977]:
[1,1,1,3,3,3][3:]

[3, 3, 3]

In [1007]:
def clean(row):
    if len(row['extra'])==6:
        if (row['extra'][-1] in row['massless'] 
            and (row['solution']+row['extra'][3:]).count(  row['extra'][-1] )>3
            and row['solution'].count(row['ll'])<=2
           ):
            return False

In [1012]:
tm=tm[tm.apply(clean,axis='columns').fillna(True)].reset_index(drop=True)

In [1013]:
#assert tm[tm['sltn']=='[1, 1, -3, -4, 6, 6, -7]'].iloc[-1]['massless']==[-7]

In [1014]:
kk=tm[['nn','l','k','solution','gcd','extra','ll','ν','s','massless','sp']].copy()

In [1015]:
import re
def add_boldsymbol(ss):
    if str(ss).find(r'\boldsymbol')==-1:
        return re.sub('(\-*[0-9]+)',r'\\boldsymbol{\1}',str(ss))
    else:
        return ss

In [1016]:
kk=tm[['nnn','l','k','solution','gcd','extra','ll','ν','s','massless','sp']].copy()

for i in kk.index:
    #for s in ['DD','DM','XD','XM']:
    if not kk.loc[i,'massless']:
        print(i)
        kk.loc[i,'nnn']=add_boldsymbol(kk.loc[i,'nnn'])
        #kk.loc[i,s]  =add_boldsymbol(kk.loc[i,s])

kkk=kk#[['n','l','k','solution','gcd']]#Ref','DD','DM','XD','XM']]
def f(x):
    return  r'{}'.format(str(x).replace('[','(').replace(']',')'))

kkk.to_latex('solutions.tex',index=False,formatters=dict( [(k,f) for k in kk.columns ]) ,escape=False  )

21
30


In [1017]:
cat solutions.tex

\begin{tabular}{llllrlrrrll}
\toprule
             nnn &                l &                 k &                                solution & gcd &                        extra &  ll &   ν &  s &        massless &    sp \\
\midrule
               8 &          (1, -2) &           (-4, 1) &                  (1, -4, -4, 9, 9, -11) &   3 &               (1, 1, -1, -1) &  -4 &   9 & -5 &           (-11) &    22 \\
               9 &          (-1, 1) &       (-1, 0, -1) &                (1, 2, 2, -3, -3, -3, 4) &   1 &               (1, 1, -1, -1) &   2 &  -3 &  1 &             (4) &     8 \\
               9 &           (3, 1) &       (-1, -5, 7) &                (2, 2, -4, 7, -8, -8, 9) &   1 &               (-4, -4, 4, 4) &   2 &  -8 &  6 &          (7, 9) &    16 \\
              10 &          (-1, 1) &       (-1, 0, -1) &                (1, 2, 2, -3, -3, -3, 4) &   1 &        (-4, -4, -4, 4, 4, 4) &  -3 &   2 &  1 &          (1, 4) &     5 \\
              10 &      (0, -3, -1) &  (-3, -5, 

In [1018]:
kkk[kkk['solution'].astype(str)=='[1, 1, 2, 2, 4, -5, -7, -7, 9]']

,nnn,l,k,solution,gcd,extra,ll,ν,s,massless,sp
11,11,"[1, 2, 1, 2]","[-5, -3, -6, -3]","[1, 1, 2, 2, 4, -5, -7, -7, 9]",1,"[-5, -5, 5, 5]",1,-7,6,[9],18


In [1019]:
kkk[kkk['solution'].astype(str)=='[1, -2, -2, 4, 5, -7, -7, 8]']

,nnn,l,k,solution,gcd,extra,ll,ν,s,massless,sp
10,11,"[-1, 0, 1]","[-4, -2, 2]","[1, -2, -2, 4, 5, -7, -7, 8]",1,"[-16, -16, -16, 16, 16, 16]",-7,-2,9,[16],32


In [1020]:
kkk[kkk['solution'].astype(str)=='[4, 4, 4, -5, -9, -10, -10, 11, 11]']

,nnn,l,k,solution,gcd,extra,ll,ν,s,massless,sp


In [1021]:
kkk

,nnn,l,k,solution,gcd,extra,ll,ν,s,massless,sp
0,8,"[1, -2]","[-4, 1]","[1, -4, -4, 9, 9, -11]",3,"[1, 1, -1, -1]",-4,9,-5,[-11],22
1,9,"[-1, 1]","[-1, 0, -1]","[1, 2, 2, -3, -3, -3, 4]",1,"[1, 1, -1, -1]",2,-3,1,[4],8
2,9,"[3, 1]","[-1, -5, 7]","[2, 2, -4, 7, -8, -8, 9]",1,"[-4, -4, 4, 4]",2,-8,6,"[7, 9]",16
3,10,"[-1, 1]","[-1, 0, -1]","[1, 2, 2, -3, -3, -3, 4]",1,"[-4, -4, -4, 4, 4, 4]",-3,2,1,"[1, 4]",5
4,10,"[0, -3, -1]","[-3, -5, -6, -4]","[3, -4, -6, -6, 7, 7, 8, -9]",1,"[8, 8, -8, -8]",7,-6,-1,[-9],18
5,10,"[-4, -1, 1]","[-13, -14, -7]","[1, 1, -2, -4, -4, 10, 11, -13]",2,"[-2, -2, 2, 2]",1,-4,3,[11],22
6,10,"[0, 2, 3]","[-1, -7, -4, -5]","[1, -5, -5, -10, 11, 11, 11, -14]",2,"[1, 1, -1, -1]",-5,11,-6,"[-10, -14]",24
7,10,"[-1, 0, 1]","[-11, -8, 8]","[1, 2, 13, 13, -16, -16, -19, 22]",2,"[-19, -19, 19, 19]",-16,13,3,[22],44
8,10,"[9, -6]","[-5, -10, 8]","[9, 9, -24, -24, -24, 25, 29]",45,"[-39, -39, -39, 39, 39, 39]",-24,9,15,"[25, 29]",54
9,11,"[0, 1, -2]","[-1, -2, -4, 1]","[2, 2, 2, 2, -5, -5, -5, 7]",2,"[-8, -8, -8, 8, 8, 8]",-5,2,3,[7],14


In [419]:
kkk[kkk['solution'].astype(str)=='[1, -2, -2, 4, 5, -7, -7, 8]']

,nnn,l,k,solution,gcd,extra,ll,ν,s,massless,sp


In [ ]:
pd.set_option('display.max_rows', 500)

In [ ]:
tm[['nn','l','k','extra','solution','gcd','ll','ν','s','massless','sp','D_msls','rptd','nl','nν','nr','M_msls']].drop_duplicates(subset=['nn','D_msls','rptd','nl','nν','nr','M_msls'])

Matrix rank

In [ ]:
import scipy as sp
import numpy as np
#For arbitrary precision see:
#https://stackoverflow.com/questions/6876377/numpy-arbitrary-precision-linear-algebra

In [ ]:
n=3
Z=np.zeros((n,n))
MD=np.random.uniform(100,200,(n,n))
MS=np.random.uniform(100,200,(n,n))
MF=np.zeros((n,n))
if n>=1:
    MF[0,0]=1E5
if n>=2:
    MF[1,1]=1E6
if n>=3:
    MF[2,2]=1E7

In [ ]:
#M=np.vstack( (np.hstack((Z,MD)),np.hstack((MS,MF))) )
M=np.vstack( (np.hstack((Z,MD)),np.hstack((MD.transpose(),MF))) )
#M=np.longdouble(M)
M.round(1)

In [ ]:
#sp.linalg.eig(M)
np.linalg.eig(M)

In [ ]:
(1/137)/(2*np.pi)